In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import h5py

In [0]:
hdf5_path = '/content/drive/My Drive/similiarimagedataset_dreambot_weddingcard_300_new_classi_slider.hdf5'
# hdf5_pathtest = '/content/drive/My Drive/similiarimagedataset_dreambot_test_500.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")
# hdf5_filetest = h5py.File(hdf5_pathtest, "r")

# reshape to be [samples][pixels][width][height]
X_train = hdf5_file.get('train_img').value
# Y_train = hdf5_file.get('train_labels').value
# X_val  = hdf5_filetest.get('val_img').value
# y_val  = hdf5_filetest.get('val_labels').value
# X_test  = hdf5_filetest.get('test_img').value
# y_test  = hdf5_filetest.get('test_labels').value
# X_train = X_train.reshape(X_train.shape[2],X_train.shape[1],X_train.shape[0])
# X_test = X_test.reshape(X_test.shape[2],X_test.shape[1],X_test.shape[0])
hdf5_file.close()
X_train = X_train.reshape(X_train.shape[0], 300, 300,3).astype('float32')
# X_test = X_test.reshape(X_test.shape[0], 500, 500,3).astype('float32')
# X_val  = X_val.reshape(X_val.shape[0], 299, 299,3).astype('float32')

In [0]:
# X_train = X_train.astype('uint32')/255
X_train_list = []
X_train_resized = []
for i in range(len(X_train)):
  img = cv2.resize(X_train[i], (256, 256), interpolation=cv2.INTER_CUBIC)
  X_train_resized.append(img)
X_train_resized = np.array(X_train_resized)
imagedatagen = ImageDataGenerator(samplewise_center=True,samplewise_std_normalization=True)
# for i in range(len(X_train)):
imagedatagen.fit(X_train_resized)
batchgen = imagedatagen.flow(X_train_resized,batch_size=769,shuffle=False)
X_train = batchgen.next()
plt.imshow(X_train[0])



# hdf5_path = '/content/drive/My Drive/similiarimagedataset_dreambot_weddingcard_256new2.hdf5'
# train_addrs = X_train
# # elif data_order == 'tf':
# train_shape = (len(train_addrs), 256, 256,3)
# #     val_shape = (len(val_addrs), 299, 299,3)
# #     test_shape = (len(test_addrs), 299, 299,3)QA
# # open a hdf5 file and create earrays
# hdf5_file = h5py.File(hdf5_path, mode='w')
# hdf5_file.create_dataset("train_img", train_shape, np.int32)
# hdf5_file.create_dataset("train_labels", (len(train_addrs),12), np.int32)
# hdf5_file["train_labels"][...] = Y_train

#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# #   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#   hdf5_file["train_img"][i, ...] = img[None]
# # X_train = X_train.astype('float32')/255.0
# # X_tests  = X_test.astype('float32')/255.0
# # for i,j in X_train:
# #     mean = np.mean(i, axis=(0, 1, 2))
# # #   print(mean)
# #     i -= mean
# # for i in X_test:
# #   mean = np.mean(i, axis=(0, 1, 2))
# #   i -= mean
# hdf5_file.close()

In [0]:
plt.imshow(X_train[0].reshape(300,300,3).astype('int32'))

In [0]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("elu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("elu")(x)
    return x

In [0]:
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
#     u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
#     u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
#     u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
#     u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(3, (1, 1), activation='tanh') (c9)
    
    
#     flatten = Flatten()(c5)
#     x   = Dense(500, activation='relu')(flatten)
#     x   = Dense(250, activation='relu')(x)
#     x   = Dense(50, activation='relu')(x)
#     dense_layer_output   = Dense(12, activation='hard_sigmoid')(x)

    model = Model(inputs=[input_img], outputs=[outputs])#,dense_layer_output])
    return model

In [0]:
input_img = Input((256, 256, 3), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)

model.compile(optimizer=Adam(), loss="mean_squared_error")#,"mean_squared_error"])
model.summary()

In [0]:
model.fit(X_train,X_train,batch_size=30,epochs=200,shuffle='batch')

In [0]:
try:
    model_json = model.to_json()
    with open("/content/drive/My Drive/similiarautoencoder_final_256new_unet_onlyautoencoder_use_slider.json", "w") as json_file:
        json_file.write(model_json)
except:
    pass
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/similiarautoencoder_final_256new_unet_onlyautoencoder_use_slider.h5")
print("Saved model to disk")

In [0]:
plt.imshow(model.predict(X_train[:1]).reshape(256,256,3))

In [0]:
x = model.get_layer('conv2d_31').output
flatten = Flatten()(x)
encoder = Model(inputs=model.input, outputs=flatten)

In [0]:
try:
    model_json = encoder.to_json()
    with open("/content/drive/My Drive/similiarautoencoder_encoder_final_256new_unet_only_autoencoder_use_slider.json", "w") as json_file:
        json_file.write(model_json)
except:
    pass
# serialize weights to HDF5
encoder.save_weights("/content/drive/My Drive/similiarautoencoder_encoder_final_256new_unet_only_autoencoder_use_slider.h5")
print("Saved model to disk")